In [ ]:
"""
Chapter 14 Reduction operators In Processes:

pd.set_options('display.max_rows', 2000)
pd.set_options('display.max_columns', 2000)
pd.set_options('display.width', 1000)
"""

In [ ]:
import os

import requests

import json

import multiprocessing

import msgpack


import numpy as np

import pandas as pd

from dotenv import load_dotenv

import alpaca_trade_api as tradeapi

from math import sqrt

from timeit import default_timer as timer

import concurrent
import time

# Load the environment variables from the .env file
#by calling the load_dotenv function

load_dotenv()

In [ ]:
"""
class ReducttionConsumer(multiprocessing.Process):
    def __init__(self, task_queue, result_queue):
        multiprocessing.Process.__init__(self)
        self.task_queue = task_queue
        self.result_queue = result_queue
        
    def run(self):
        pname = self.name
        print("Using process %s.." % pname)
        
        while True:
            num1 = self.task_queue.get()
            
            if num1 is None:
                print("Exiting process %s" % pname)
                self.task_queue.task_queue()
                break
                
            self.task_queue.task_done()
            num2 = self.task_queue.get()
            if num2 is None:
                print("Reaching the end with process %s and number %i." % (pname, num1))
                self.task_queue.task_done()
                self.result_queue.put(num1)
                break
                
            print("Running process %s on numbers %i and %i." % (pname, num1, num2))
            self.task_queue.task_done()
            self.result_queue.put(num1 + num2)
            
def reducesum(array):
    tasks = multiprocessing.JoinableQueue()
    results = multiprocessing.JoinableQueue()
    result_size = len(array)
    
    n_consumers = multiprocessing.cpu_count()
    
    for item in array:
        results.put(item)
        
    while result_size > 1:
        tasks = results
        results = multiprocessing.JoinableQueue()
        
        consumers = [ReducttionConsumer(tasks, results) for i in range(n_consumers)]
        for consumer in consumers:
            consumer.start()
            
        for i in range(n_consumers):
            tasks.put(None)
            
        tasks.join()
        result_size = result_size // 2 + (result_size % 2)
        #print('-' * 40)
        
    return results.get()



my_array = [i for i in range(20)]

result = reducesum(my_array)

"""

In [ ]:

# Concurrent Image Processing 



In [ ]:

import cv2


In [ ]:

im = cv2.imread('testim.jpg')


In [ ]:
name = 'javier'

cv2.imshow(name, im)
cv2.waitKey(0)

print(im)
print('Type:', type(im))
print('Shape:', im.shape)
print("Top-left pixel:", im[0, 0])

print('Done.')

In [ ]:
# Grayscaling

import  cv2 


im = cv2.imread('Kuba_R.jpg')
gray_im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

cv2.imshow("Grayscale", gray_im)

cv2.waitKey(0)

print(gray_im)
print("Type:", type(gray_im))
print("Shape:", gray_im.shape)
cv2.imwrite("Kuba_Gray.jpg", gray_im)

In [ ]:
# Theresholding 
# Thhe key is finding an appropriate threshold
# so that the pixels in an image are segmented in a way tha allow 
# separate regions in the image to become more obvious


ret, custom_thresh_im = cv2.threshold(gray_im, 127, 255, cv2.THRESH_BINARY)

In [ ]:
cv2.imwrite("custom_Kuba.jpg", custom_thresh_im)

In [ ]:
im = cv2.imread('custom_Kuba.jpg')
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

mean_thresh_im = cv2.adaptiveThreshold(im, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 11, 2)

cv2.imwrite("mean_custom_Kuba.jpg", mean_thresh_im)


In [ ]:

gauss_thresh_im = cv2.adaptiveThreshold(im, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 11, 2)

cv2.imwrite('gauss_thresh_kuba.jpg', gauss_thresh_im)

In [ ]:
# Applying concurrenct to image processing 

# implemeting a program that can concurrently process these images using thresholding.


In [ ]:
'''
from multiprocessing import Pool
import cv2
import sys
from timeit import default_timer as timer

THRSH_METHOD = cv2.ADAPTIVE_THRESH_GAUSSIAN_C

INPUT_PATH = 'large_input/'

OUT_PATH = 'large_out/'

n = 3

names  = ['rug_%i_%i.jpg' % (i, j) for i in range(n) for j in range(n)]

def process_threshold(im, output_name, threshold_method):
    gray_ims = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    thresh_im = cv2.adaptiveThreshold(gray_ims, 255, threshold_method, cv2.THRESH_BINARY, 11, 2)
    
    cv2.imwrite(OUT_PATH + output_name, thresh_im)
    
    
if __name__ == '__main__':
    for n_processes in range(1, 9):
        start = timer()
        
        with Pool(n_processes) as p:
            p.starmap(process_threshold, [(cv2.imread(INPUT_PATH + name), name, THRSH_METHOD) for name in names])
            
        print('Took %4.f seconds with %i process(es), ' % (timer() - start, n_processes))
    
    print("Done..")
    
    
    

from multiprocessing import Pool
import cv2
import sys
from functools import partial
from timeit import default_timer as timer

THRSH_METHOD = cv2.ADAPTIVE_THRESH_GAUSSIAN_C

INPUT_PATH = 'large_input/'

OUT_PATH = 'large_out'

n = 3

names  = ['rug_%i_%i.jpg' % (i, j) for i in range(n) for j in range(n)]

def process_threshold(im, output_name, threshold_method):
    gray_ims = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    thresh_im = cv2.adaptiveThreshold(gray_ims, 255, threshold_method, cv2.THRESH_BINARY, 11, 2)
    
    cv2.imwrite(OUT_PATH + output_name, thresh_im)
    
if __name__ == '__main__':
    for n_processes in range(1, 9):
        start = timer()
        
        with Pool(n_processes) as p:
            p.map(partial(process_threshold, thresh_method = THRSH_METHOD), names)
            print("Took %.4f seconds with %i process(es)." % (timer() - start, n_processes))
            
from multiprocessing import Pool
import cv2
import sys
from functools import partial
from timeit import default_timer as timer

THRSH_METHOD = cv2.ADAPTIVE_THRESH_GAUSSIAN_C

INPUT_PATH = 'large_input/'

OUT_PATH = 'large_out'

n = 3

names  = ['rug_%i_%i.jpg' % (i, j) for i in range(n) for j in range(n)]

def process_threshold(im, output_name, threshold_method):
    gray_ims = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    thresh_im = cv2.adaptiveThreshold(gray_ims, 255, threshold_method, cv2.THRESH_BINARY, 11, 2)
    
    cv2.imwrite(OUT_PATH + output_name, thresh_im)
    
if __name__ == '__main__':
    for n_processes in range(1, 9):
        start = timer()
        
        with Pool(n_processes) as p:
            p.map(partial(process_threshold, thresh_method = THRSH_METHOD), names)
            print("Took %.4f seconds with %i process(es)." % (timer() - start, n_processes))

'''

In [ ]:

from multiprocessing import Pool
import cv2
import sys
from functools import partial
from timeit import default_timer as timer

In [ ]:

# Using Haar Cascade model to detect faces in images.

face_cascade = cv2.CascadeClassifier('input/haarcascade_frontalface_default.xml')

In [ ]:
for filename in ['face1.jpg', 'face2.jpg', 'wayback.jpg']:
    im = cv2.imread('input/' + filename)
    gray_im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(im, scaleFactor=1.2)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(im, (x,  y), (x + w, y + h), (0, 255, 0), 2)
        
    cv2.imshow('%i face(s) found' % len(faces), im)
    cv2.waitKey(0)
    
print('Done..')

In [ ]:
"""
Introduction to Asynchronous Programming 
______
To improve responsiveness of the program we use asyncio module
    
"""

In [ ]:

from math import sqrt

def is_prime(x):
    print('Processing %i...' % x)

    if x < 2:
        print('%i is not a prime number.' % x)

    elif x == 2:
        print('%i is a prime number.' % x)

    elif x % 2 == 0:
        print('%i is not a prime number.' % x)

    else:
        limit = int(sqrt(x)) + 1
        for i in range(3, limit, 2):
            if x % i == 0:
                print('%i is not a prime number.' % x)
                return

        print('%i is a prime number.' % x)

if __name__ == '__main__':

    is_prime(9637529763296797)
    is_prime(427920331)
    is_prime(157)


In [ ]:
from math import sqrt
import asyncio

async def is_prime(x):
    print('Processing %i..' % x)
    
    if x < 2:
        print('%i is not a prime number')
        
    elif x == 2:
        print('%i  is a prime number.' % x)
        
    elif x % 2 == 0:
        print('%i is not a prime number.' % x)
        
    else:
        limit = int(sqrt(x)) + 1
        for i in range(3, limit, 2):
            if x % i == 0:
                print('%i is not a prime number.' % x)
                return
            elif i % 100000 == 1:
                await asyncio.sleep(1)
                
        print('%i is a prime number.' % x)
    

In [ ]:
    
async def main():
    task1 = loop.create_task(is_prime(5456415645641654))
    
    task2 = loop.create_task(is_prime(2313641564))

    task3 = loop.create_task(is_prime(6416))
    
    await asyncio.wait([task1, task2, task3])
    
    


In [ ]:
    
if __name__ == '__main__':
    try:
        loop =  asyncio.get_event_loop()
        loop.run_until_complete(main())
    except Exception as e:
        print('There was a problem:')
        print(str(e))
    finally:
        loop.close()


In [ ]:
# Chapter 17 Implementing Asynchoronous Programming in Py

# Asynchoroncously counting down

import time

def count_down(name, delay):
    indents = (ord(name) - ord('A')) * '\t'
    
    n = 3 
    while n:
        time.sleep(delay)
        
        duration = time.perf_counter() - start
        print('-' * 40)
        print('%.4f \t%s%s = %i' % (duration, indents, name, n))
        
        n -= 1
        
start = time.perf_counter()

count_down('A', 1)
count_down('B', 0.8)
count_down('C', 0.5)

print('-' * 40)
print('Done.')

In [ ]:
from math import sqrt
import asyncio


In [ ]:
async def count_down(name, delay):
    indents = (ord(name) - ord('A')) * '\t'
    
    n = 3
    while n:
        await asyncio.sleep(delay)
        
        duration = time.perf_counter() - start
        print('-' * 40)
        print('%.4f \t%s%s = %i' % (duration, indents, name, n))

        

In [ ]:
loop = asyncio.get_event_loop()

tasks = [
    loop.create_task(count_down('A', 1)),
    
    loop.create_task(count_down('B', 0.8)),
    
    loop.create_task(count_down('C', 0.5))
]

start = time.perf_counter()
loop.run_until_complete(asyncio.wait(tasks))

print('-' * 40)
print('Done')

In [ ]:
# Asynchoronous prime-checking

from math import sqrt

def is_prime(x):
    print('Processing %i...' % x)
    if x < 2:
            print('%i is not a prime number')
        
    elif x == 2:
        print('%i  is a prime number.' % x)
        
    elif x % 2 == 0:
        print('%i is not a prime number.' % x)
        
    else:
        limit = int(sqrt(x)) + 1
        for i in range(3, limit, 2):
            if x % i == 0:
                print('%i is not a prime number.' % x)
                return
        print('%i is a prime number' % x)
        
        
        
if __name__ == '__main__':
    
    is_prime(9637529763296797)
    is_prime(427920331)
    is_prime(157)


In [ ]:
async def main():
    task1 = loop.create_task(is_prime(5456415645641654))
    
    task2 = loop.create_task(is_prime(2313641564))

    task3 = loop.create_task(is_prime(6416))
    
    await asyncio.wait([task1, task2, task3])
    
    
    

In [2]:
from math import sqrt 
import asyncio

async def is_prime(x):
    print('Processing %i...' % x)
    
    if x < 2:
        print('%i is not a prime number' % x)
        
    elif x % 2:
        print('%i is not prime' % x)
        
    else:
        limit = int(sqrt(x)) + 1
        for i in range(3, limit, 2):
            if x % i == 0:
                print('i is not a prime number.' % x)
                return
            elif i % 100000 == 1:
                await asyncio.sleep(0)
                
        print('%i is a prime number' % x)
        
        
try:
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
except Exception as e:
    print('There was a problem:')
    print(str(e))
finally:
    loop.close()

There was a problem:
name 'main' is not defined


RuntimeError: Cannot close a running event loop